# Базовая часть

Рассмотрим задание 2 из лекции. При анализе отзывов на работу техподдержки вы считали массив из отзывов (для простоты отзыва приведено всего два, но могло быть и больше):

In [1]:
reviews = ["""Телефон абонента: +7 916 000 00 01
Время: 10:34
Дата: 20.05.2017
Оценка: 1
Длительность разговора (мин.): 0:32
Комментарий: Общение с сотрудником вашей техподдержки мне не понравилось. Раньше всегда отвечала девушка с приятным голосом, а сегодня какой-то мужчина. Совсем никуда не годится, примите меры!!!""",
"""Телефон абонента: +7 916 000 00 02
Время: 9:20
Дата: 21.05.2017
Оценка: 5
Длительность разговора (мин.): 25:07
Комментарий: С моей проблемой мне помогли, большое спасибо!"""]

Сформируйте на основе этого массива словарь my_base, в котором по ключу в виде строки с телефоном абонента будет храниться другой словарь. Внутренний словарь по ключам в виде названий полей отзыва должен хранить значение полей (в виде строк). Например, код ниже должен выдать True:

In [ ]:
my_base['+7 916 000 00 01']['Дата'] == '20.05.2017'

Обратите внимание, что вам могут немного мешать пробелы после знаков ":"

#### Вариант №1 (явный)

In [2]:
my_base = {}
for info in reviews:
    strings = info.split('\n')
    main_key = strings[0].split(': ')[1]
    internal_dict = {}
    for param in strings[1:]:
        key, value = param.split(': ')
        internal_dict[key] = value
    my_base[main_key] = internal_dict
my_base['+7 916 000 00 01']['Дата'] == '20.05.2017'

True

#### Вариант №2 (с использованием генератора словарей)

In [3]:
my_base = {}
for info in reviews:
    strings = info.split('\n')
    my_base[strings[0].split(': ')[1]] = {param.split(': ')[0]: param.split(': ')[1] for param in strings[1:]}
my_base['+7 916 000 00 01']['Время'] == '10:34'

True

#### Вариант №3 (однострочный, запутанный, неудобный в сопровождении)

In [4]:
my_base = {info.split('\n')[0].split(': ')[1]: {param.split(': ')[0]: param.split(': ')[1] for param in info.split('\n')[1:]} for info in reviews}
my_base['+7 916 000 00 02']['Оценка'] == '5'

True

# Продвинутая часть

Попробуйте выполнить предыдущее задание, но теперь обеспечив хранение полей из отзыва в словаре в тех типах данных, которые на ваш взгляд наиболее уместны. Подумайте, что делать с датой и временем. Возможно вам покажется полезным немного воспользоваться поисковиком или документацией к Python, а может быть вам хватит стандартных типов данных.
Объясните свой выбор в комментариях к коду или markdown ячейках и продемонстрируйте, что поля имеют нужный тип.

#### Ответ:
В python среди стандартных модулей есть модуль datetime для работы с датами и временем. Приведя время и дату к типу данных datetime, у нас появится возможность производить всяческие вычисления с датой, а также достаточно легко задавать любой формат отображения даты и времени без изобретения собственных "велосипедов" по парсингу строк с датами.

Произведем преобразование всех значений в словаре, которые так или иначе связаны с временем в тип datetime.

In [5]:
# импортируем необходимые классы из модуля
from datetime import datetime, timedelta

# пробегаемся циклом по словарю и производим преобразование типов и переприсваивание значений у соответствующих ключей
for subs in my_base:
    my_base[subs]['Дата'] = datetime.strptime(my_base[subs]['Дата'], '%d.%m.%Y')
    hrs, mins = my_base[subs]['Время'].split(':')
    my_base[subs]['Время'] = timedelta(hours=int(hrs), minutes=int(mins))
    min_dur, sec_dur = my_base[subs]['Длительность разговора (мин.)'].split(':')
    my_base[subs]['Длительность разговора (мин.)'] = timedelta(minutes=int(min_dur), seconds=int(sec_dur))

Убедимся в том, что данные теперь хранятся с типом datetime:

In [6]:
for subs in my_base:
    print type(my_base[subs]['Дата']), type(my_base[subs]['Время']), type(my_base[subs]['Длительность разговора (мин.)'])

<type 'datetime.datetime'> <type 'datetime.timedelta'> <type 'datetime.timedelta'>
<type 'datetime.datetime'> <type 'datetime.timedelta'> <type 'datetime.timedelta'>


В качестве примера работы с датами придумал следующую задачу:

    Необходимо вывести на экран данные о всех абонентах, разговор техподдержки с которыми закончился не позднее 5 суток от текущего времени. В выводе должны присутствовать номер абонента, а также время начала и окончания вызова в формате dd/mm/yy hh:mm:ss.

In [7]:
for subs in my_base:
    date = my_base[subs]['Дата']
    time = my_base[subs]['Время']
    duration = my_base[subs]['Длительность разговора (мин.)']
    if datetime.now() < date + time + duration + timedelta(days=5):
        start_call_datetime = (date + time).strftime('%d/%m/%y %H:%M:%S')
        end_call_datetime = (date + time + duration).strftime('%d/%m/%y %H:%M:%S')
        print 'Время на момент выборки:', datetime.now().strftime('%d/%m/%y %H:%M:%S')
        print '-'*60
        print 'Абонент:\t\t\t| %s' %subs
        print 'Дата и время начала вызова:\t| %s' %start_call_datetime
        print 'Дата и время окончания вызова:\t| %s' %end_call_datetime
        print '-'*60

Время на момент выборки: 25/05/17 13:25:45
------------------------------------------------------------
Абонент:			| +7 916 000 00 02
Дата и время начала вызова:	| 21/05/17 09:20:00
Дата и время окончания вызова:	| 21/05/17 09:45:07
------------------------------------------------------------


# Сложная часть

На лекции было рассказано про неизменяемые типы, в частности про tuple. Подумайте, можно ли "сломать" неизменяемость tuple? Как сделать tuple изменяемым? Можно ли будет его добавить в множество или использовать в качестве ключа в словаре?

### Ответ:
    "Сломать" можно, если в качестве одного из элементов кортежа использовать изменяемый тип данных. В случае попытки использовать такой кортеж в качестве ключа для словаря или же во множестве, получим ошибку "TypeError: unhashable type":

#### Пример №1 (со списком внутри кортежа)

In [8]:
my_list = [2, 4]
my_tuple = (1, my_list, 3)
print my_tuple
my_list.append(6)
print my_tuple

(1, [2, 4], 3)
(1, [2, 4, 6], 3)


In [9]:
test_dict = {my_tuple: 'test'}

TypeError: unhashable type: 'list'

In [10]:
test_set = set(my_tuple)

TypeError: unhashable type: 'list'

#### Пример №2 (с множеством внутри кортежа)

In [11]:
my_set = set('123')
my_tuple2 = (1, my_set, 3)
print my_tuple2
my_set.add('4')
print my_tuple2

(1, set(['1', '3', '2']), 3)
(1, set(['1', '3', '2', '4']), 3)


In [12]:
test_dict = {my_tuple2: 'test'}

TypeError: unhashable type: 'set'

In [13]:
test_set = set('135')
test_set.add(my_set)

TypeError: unhashable type: 'set'